In [2]:
from cv2 import AGAST_FEATURE_DETECTOR_AGAST_5_8
from src.getter import Get
import os
from models.sqliteModels import generate_symbols


In [3]:
symbol = generate_symbols()

In [9]:
stocks = [next(symbol) for i in range(100)]


# Main Code: 

In [3]:
from models.mysql_db import generate_symbols
from src.pricehistory import price_history